# Import Libraries

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


The code imports necessary libraries, including Pandas, NumPy, Matplotlib, TensorFlow, and Keras. It also sets up a deep learning model using the ResNet50 architecture for image classification with data augmentation using ImageDataGenerator.

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense




# Preprocessing training and testing data

## preprocessing training data

This code initializes an image data generator with pixel rescaling and data augmentation settings. It then loads the training data from the specified directory, resizes the images to 224x224 pixels, and applies the configured augmentation.

In [3]:

train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

training_set = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/DL/brain-tumor-classification-mri/Training',
    target_size=(224, 224),
    batch_size=32,
    class_mode='sparse',
    shuffle=True)

Found 2870 images belonging to 4 classes.


## Preprocessing testing data

This code initializes a test data generator with pixel rescaling and similar augmentation settings. It then loads the testing data from the specified directory, resizes the images to 224x224 pixels, and applies the configured augmentation.


In [4]:
test_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

testing_set = test_datagen.flow_from_directory(
    '/content/drive/MyDrive/DL/brain-tumor-classification-mri/Testing',
    target_size=(224, 224),
    batch_size=32,
    class_mode='sparse',
    shuffle=True)

Found 394 images belonging to 4 classes.


# Train the dataset

The pre-trained ResNet50 model is used as a base, followed by a global average pooling layer, a dense layer with 256 units and ReLU activation, and a final dense layer with softmax activation for multi-class classification.

In [5]:
# resnet50
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
cnn = Sequential()
cnn.add(base_model)
cnn.add(GlobalAveragePooling2D())
cnn.add(Dense(units=256, activation='relu'))
cnn.add(Dense(units=4, activation='softmax'))

94765736/94765736 [==============================] - 4s 0us/step


This code sets the last 8 layers of the pre-trained ResNet50 base model as trainable, allowing fine-tuning for the specific task in this notebook. This selective training modification can enhance model performance by adapting the pre-trained weights to the new dataset while retaining knowledge from earlier layers

In [6]:
for layer in base_model.layers[:-8]:
    layer.trainable = True

# Compile cnn with a custom learning rate

This code defines an Adam optimizer with a learning rate of 0.0001 for the previously defined CNN model. The model is compiled using sparse categorical crossentropy as the loss function and accuracy as the metric. The training set is used to fit the model with a batch size of 32, and validation data from the testing set is utilized for model evaluation over 20 epochs in the notebook.

In [7]:

opt = tf.keras.optimizers.Adam(learning_rate=0.0001)
cnn.compile(optimizer=opt, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Fit the model
cnn.fit(x=training_set, batch_size=32, validation_data=testing_set, epochs=20)

Epoch 1/20
90/90 [==============================] - 1968s 21s/step - loss: 0.3600 - accuracy: 0.8610 - val_loss: 3.3611 - val_accuracy: 0.2665
Epoch 2/20
90/90 [==============================] - 62s 692ms/step - loss: 0.1694 - accuracy: 0.9394 - val_loss: 5.5351 - val_accuracy: 0.2665
Epoch 3/20
90/90 [==============================] - 63s 694ms/step - loss: 0.0869 - accuracy: 0.9725 - val_loss: 3.6383 - val_accuracy: 0.2690
Epoch 4/20
90/90 [==============================] - 66s 733ms/step - loss: 0.0639 - accuracy: 0.9801 - val_loss: 7.2172 - val_accuracy: 0.2665
Epoch 5/20
90/90 [==============================] - 62s 681ms/step - loss: 0.0462 - accuracy: 0.9868 - val_loss: 4.7342 - val_accuracy: 0.2665
Epoch 6/20
90/90 [==============================] - 63s 691ms/step - loss: 0.0367 - accuracy: 0.9902 - val_loss: 5.3294 - val_accuracy: 0.2690
Epoch 7/20
90/90 [==============================] - 63s 701ms/step - loss: 0.0585 - accuracy: 0.9822 - val_loss: 3.9802 - val_accuracy: 0.3426

# Evaluate accuracy on the test set

This code evaluates the trained CNN model on the testing set and prints the test accuracy in the notebook.

In [8]:

accuracy = cnn.evaluate(testing_set)[1]
print("Test Accuracy:", accuracy)

13/13 [==============================] - 6s 467ms/step - loss: 1.5313 - accuracy: 0.8122
Test Accuracy: 0.8121827244758606


# Print model summary

This code block prints the summary of the model

In [9]:

cnn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 7, 7, 2048)        23587712  
                                                                 
 global_average_pooling2d (  (None, 2048)              0         
 GlobalAveragePooling2D)                                         
                                                                 
 dense (Dense)               (None, 256)               524544    
                                                                 
 dense_1 (Dense)             (None, 4)                 1028      
                                                                 
Total params: 24113284 (91.98 MB)
Trainable params: 24060164 (91.78 MB)
Non-trainable params: 53120 (207.50 KB)
_________________________________________________________________
